<h2>Convolutional Neural Network in Keras</h2>
<p>Train a simple convnet on the MNIST dataset.</p>
<p>Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python mnist_cnn.py
Get to 99.25% test accuracy after 12 epochs (there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU</p>
<p>
For this tutorial, a convolutional neural network (CNN) is built using Keras. It is trained and tested using the MNIST handwritten digits dataset. The CNN consists of multiple layers of convolution and max pooling, ending with a fully connected MLP for classification. 
</p>
<p>
This example is pulled from the examples on the Keras Github repository. It has been converted into an iPython notebook for ease of use and shareability. It has also been modified slightly to explore flexibility in hyperparameters.
</p>

In [1]:
from __future__ import print_function
import numpy as np

In [2]:
np.random.seed(1337) # for reproducibility of initial weights

In [3]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils

Using Theano backend.


In [4]:
# Batch size for stochastic gradient descent; e.g. number of samples per run
batch_size = 128
# Output number of classes. MNIST has 10 possible classes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
nb_classes = 10
# Number of iterations over the entire dataset when training
nb_epoch = 12

In [5]:
# input image dimensions MNIST
img_rows, img_cols = 28, 28
# number of convolutional filters to use, can be different for multiple convolutional layers
nb_filters1 = 32
nb_filters2 = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size , can vary for different layers
nb_conv1 = 3 # (3x3 covolution)
nb_conv2 = 4 # (4x4 covolution)

In [6]:
# the data, shuffled and split between tran and test sets, may have issues with proxy/firewall here. 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [7]:
# Reshape to include a 4th dimension, such that the dataset is tupled as (num_samples, num_bands, img_num_rows, img_num_cols)
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalize the training set to a value between 0 and 1
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [8]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [9]:
# Initialize an empty model
model = Sequential()

# First layer: Convolution2D - Generates 32 feature maps, using a 3x3 convolution filter
model.add(Convolution2D(nb_filters1, nb_conv1, nb_conv1, border_mode='valid', input_shape=(1, img_rows, img_cols)))
# For each node, sum the input x weights, and run Rectified Linear Unit (ReLu) activation function. Can also use
# tanh, sigmoid, softplus, relu, hard_sigmoid, linear. The softmax activation is also available, but only makes sense
# to use this activation for output, as this is probability of classification.
model.add(Activation('relu'))
# Run max pooling using a 2x2 pooling filter, storing the maximum value
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
# Second Convolution layer - Generates 32 feature maps using a 4x4 convolution filter
model.add(Convolution2D(nb_filters2, nb_conv2, nb_conv2))
# Activation for second convolution step
model.add(Activation('relu'))
# Second max pooling step
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
# Dropout is used as a percentage of inputs to exclude during backpropagation, gradient updates. Here, 25% of the
# input units are "dropped" and not updated during backprop. This is to help prevent overfitting.
model.add(Dropout(0.25))

# After convolutions/max pooling, the features extracted can then be passed through a classification algorithm.
# A common approach is to simply the features through a fully connected layer before classifying using softmax.

# Convert the features into a single dimension vector
model.add(Flatten())

# Add a fully connected hidden layer of 128 nodes - This can be modified as a hyperparameter for testing various models
model.add(Dense(128))
# ReLu activation function for hidden layer
model.add(Activation('relu'))
# Dropout percentage for hidden layer
model.add(Dropout(0.5))
# Output layer, fully connected to 10 nodes, for each possible class (0-9)
model.add(Dense(nb_classes))
# Softmax is an activation function that converts the values to a probability for that particular class. 
# A generalization of the logistic function 
model.add(Activation('softmax'))

In [10]:
# Compile the model, using the RMSprop optimizer, and a the categorical cross entropy loss function.

# ADADelta is a variant of stochastic gradient descent. A per-dimension learning-rate method that adapts over time, 
# requires no manual parameter tuning

# Categorical_crossentropy is used with softmax to determine the N-category cross entropy of the predicted vs. 
# target variable category. Also known as multiclass logloss.
# Many additional loss functions are available, including mean_squared_error / mse, root_mean_squared_error / rmse
# mean_absolute_error / mae, mean_absolute_percentage_error / mape, mean_squared_logarithmic_error / msle, squared_hinge
# hinge, binary_crossentropy: Also known as logloss., categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adadelta')

In [11]:
# Begin Training the model
#
# Pass the training set: input and targets
# batch_size: size of the mini batch, or number of samples to run at once, including gradient updates, 
# rather than run the entire dataset
# nb_epoch: number of epochs or iterations over the entire dataset
# show_accuracy: whether or not to display the accuracy for each epoch while training
# verbose: how much detail to display, 0 - No output, 1 - More detail, 2 - Less detail
# validation_data: Dataset the model is validated against, the output displays the loss and accuracy 
# of the validation set
#
# The loss function should be minimized. Accuracy is a percentage, e.g. ~1.0 yields 100% accuracy
%time model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, \
                verbose=2, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
447s - loss: 0.2541 - acc: 0.9205 - val_loss: 0.0531 - val_acc: 0.9822
Epoch 2/12
454s - loss: 0.0973 - acc: 0.9702 - val_loss: 0.0414 - val_acc: 0.9869
Epoch 3/12
447s - loss: 0.0706 - acc: 0.9790 - val_loss: 0.0322 - val_acc: 0.9895
Epoch 4/12
446s - loss: 0.0598 - acc: 0.9817 - val_loss: 0.0311 - val_acc: 0.9892
Epoch 5/12
446s - loss: 0.0553 - acc: 0.9835 - val_loss: 0.0285 - val_acc: 0.9903
Epoch 6/12
455s - loss: 0.0504 - acc: 0.9845 - val_loss: 0.0256 - val_acc: 0.9905
Epoch 7/12
466s - loss: 0.0463 - acc: 0.9870 - val_loss: 0.0230 - val_acc: 0.9912
Epoch 8/12
459s - loss: 0.0423 - acc: 0.9870 - val_loss: 0.0223 - val_acc: 0.9921
Epoch 9/12
462s - loss: 0.0380 - acc: 0.9881 - val_loss: 0.0253 - val_acc: 0.9917
Epoch 10/12
458s - loss: 0.0364 - acc: 0.9893 - val_loss: 0.0225 - val_acc: 0.9932
Epoch 11/12
457s - loss: 0.0361 - acc: 0.9894 - val_loss: 0.0209 - val_acc: 0.9928
Epoch 12/12
457s - loss: 0.0328 - acc: 0.9897 

In [12]:
# Run the trained model on the test set. For this example, the test and validation sets are the same. This function
# is useful for running the model on a new dataset not previously seen. 
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0)

In [13]:
# print the categorical_crossentropy value of model run on the test set
print('Test score:', score[0])
# print the accuracy of the model run on the test set
print('Test accuracy:', score[1])

Test score: 0.0229262540997
Test accuracy: 0.9927
